 # eTransafe Omeprazole Heatmap

This is the use scenario that has been described in the eTox project for exploring the differences with respect to adverse events between omeprazole preclinical and clinical. In order to compute these results the following scenario is executed:
1. authenticate with the eTransafe keycloak
2. Translate the provided compound (e.g., omeprazole) to a SMILES
3. Retrieve similar compounds based on structural similarity
4. Retrieve data from the preclinical and clinical databases
5. Aggregate the data per system organ class
6. Visualize the data using a heatmap

(C) 2021 Erasmus University Medical Center, Rotterdam, The Netherlands
Author: Erik M. van Mulligen, e.vanmulligen@erasmusmc.nl

In [1]:
from knowledgehub.api import KnowledgeHubAPI
import ipywidgets as w
from IPython.display import display, Javascript
from ipypublish import nb_setup
import numpy as np
import numpy.ma as ma
import seaborn as sns
import pandas
from matplotlib.colors import LogNorm, Normalize
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
#api = KnowledgeHubAPI(server='TEST', client_secret='39c644b3-1f23-4d94-a71f-e0fb43ebd760')
api = KnowledgeHubAPI(server='DEV', client_secret='3db5a6d7-4694-48a4-8a2e-e9c30d78f9ab')

# 1. Authentication
Use eTransafe's authentication service to get access to available services

In [3]:
username = w.Text(value='erik.mulligen',placeholder='Knowledge Hub account', description='username:', disabled=False)
password = w.Password(value='', placeholder='Knowledge Hub password', description='password:', disabled=False)
loginBtn = w.Button(description='Login')
status = w.Output()

def on_button_clicked(_):
    if api.login(username.value, password.value) == False:
        print("Failed to login")
    else:
        print("successfully logged in")
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

loginBtn.on_click(on_button_clicked)
w.VBox([username, password, loginBtn])

successfully logged in


<IPython.core.display.Javascript object>

## 2. Translate compound to SMILES using semantic services
For the entered compound name, retrieve the associated SMILES using the semantic services

In [4]:
compoundName = w.Text(value='omeprazole',placeholder='compound name', description='compound:', disabled=False)
compoundBtn = w.Button(description='Retrieve')
compoundSmile = None

def on_compound_entered(_):
    global compoundSmile
    print(f'retrieving smiles for {compoundName.value}')
    compoundSmile = api.ChemistryService().getSMILESByName(compoundName.value)
    print(f'Found SMILES {compoundSmile} for {compoundName.value}')
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

compoundBtn.on_click(on_compound_entered) 
w.VBox([compoundName, compoundBtn])

retrieving smiles for omeprazole
Found SMILES ['COc1ccc2[nH]c([S+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1'] for omeprazole


<IPython.core.display.Javascript object>

## 3. Retrieve similar compounds

In [5]:
similar_compounds = api.SimilarityService().get(compoundSmile[0])

compoundNames = []
names = []
smiles = []
similarities = []

if similar_compounds != None:
    for similar_compound in similar_compounds:
        names.append(similar_compound['name'])
        smiles.append(similar_compound['smiles'])
        similarities.append(similar_compound['distance'])

pd = nb_setup.setup_pandas(escape_latex=False)
df = pd.DataFrame(np.random.rand(len(names),3),columns=['NAME','SMILES','SIMILARITY'])
df.NAME = names
df.SMILES = smiles
df.SIMILARITY = similarities
df.round(3)

,NAME,SMILES,SIMILARITY
0,esomeprazole,COc1ccc2[nH]c([S@@+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1,1.0000
1,omeprazole,COc1ccc2[nH]c([S+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1,1.0000
2,pantoprazole,COc1ccnc(C[S+]([O-])c2nc3cc(OC(F)F)ccc3[nH]2)c1OC,0.6531
3,rabeprazole,COCCCOc1ccnc(C[S+]([O-])c2nc3ccccc3[nH]2)c1C,0.6275
4,dexlansoprazole,Cc1c(OCC(F)(F)F)ccnc1C[S@@+]([O-])c1nc2ccccc2[...,0.6078
5,lansoprazole,Cc1c(OCC(F)(F)F)ccnc1C[S+]([O-])c1nc2ccccc2[nH]1,0.6078


# 4. Retrieve data from the preclinical and clinical databases

In [41]:
# studies = api.Medline().getStudiesBySMILES(smiles) + \
#           api.Faers().getStudiesBySMILES(smiles) + \
#           api.ClinicalTrials().getStudiesBySMILES(smiles) + \
studies = api.eToxSys().getStudiesByCompoundNames(names)
print(f'Found {len(studies)} studies.')

Found 8983 studies.


# 5. Aggregate the data per system organ class

Since the eTox data reports events with an organ. We use our own method to map it to MedDRA's system organ class to make it comparable

In [42]:
system = {}
all_compounds = [c.lower() for c in names]
socs = {}

api.SemanticService().getSocs(studies)
for study in studies:
    soc = study['FINDING']['__soc']
    if soc not in socs:
        socs[soc] = study['FINDING']['count']
    else:
        socs[soc] += study['FINDING']['count']


# sort the socs per count
all_socs = {k: v for k, v in sorted(socs.items(), key=lambda item: item[1], reverse=True)}


# traverse all studies and create a matrix per source
for study in studies:
    source = study['source']
    if not source in system.keys():
        system[source] = {'data':np.zeros((len(all_socs),len(all_compounds)), dtype=int).tolist(), 
                          'rows':list(all_socs.keys()), 
                          'cols':all_compounds}
    
    if object is not None:
        soc = study['FINDING']['__soc']
        row = system[source]['rows'].index(soc)
        col = system[source]['cols'].index(study['COMPOUND']['name'].lower())
        system[source]['data'][row][col] += study['FINDING']['count']
                

# 6. Visualize the data using a heatmap

Using seaborn to visualize the content of the various databases. Note that we have to think about ways to easier compare the various results.

In [70]:
services = api.Services().get()

def get_service_name(id):
    for service in services:
        if service['address'].lower().startswith('/' + id.lower()):
            return service['description']

    
i = 1
#for source in system.keys():
if True:
    source = 'etoxsyspa'
    title = get_service_name(source)
    data = dict(socs=[], findings=[], compounds=[], values=[])
    for study in studies:
        if source.startswith(study['source'].lower()):
            if study['FINDING']['finding'] is not None:                 
                data['socs'].append(study['FINDING']['__soc'])
                data['findings'].append(study['FINDING']['finding'])
                data['compounds'].append(study['COMPOUND']['name'])
                data['values'].append(study['FINDING']['count'])
    print(len(data['values']))
    fig=px.sunburst(data, path=['socs', 'findings', 'compounds'], values='values', title=title)
    fig.show()

8379
